In [ ]:
import httpx
import json

async def call_agent_api(prompt: str):
    url = "http://localhost:8000/chat"
    params = {"message": prompt}
    
    print(f"Enviando: {prompt}\n" + "-"*30)
    
    # Timeout=None é essencial para streams longos
    async with httpx.AsyncClient(timeout=None) as client:
        try:
            async with client.stream("GET", url, params=params) as response:
                if response.status_code != 200:
                    print(f"Erro na API: {response.status_code}")
                    return

                # Itera sobre cada linha do stream (SSE)
                async for line in response.aiter_lines():
                    if line.startswith("data: "):
                        # Remove o prefixo 'data: ' e carrega o JSON
                        try:
                            data = json.loads(line[6:])
                            
                            # Verifica se é um token ou o fim do stream
                            if "token" in data:
                                print(data["token"], end="", flush=True)
                            elif data.get("type") == "end":
                                print("\n" + "-"*30 + "\n[Stream Finalizado]")
                        except json.JSONDecodeError:
                            continue
        except Exception as e:
            print(f"\nErro de conexão: {e}")

In [ ]:
# Agora você verá negrito, listas e códigos formatados enquanto a IA escreve
await call_agent_api("Explique o que é o LangGraph e dê um exemplo de código em Python.")

PermissionError: [Errno 13] Permission denied: '...'